In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Input
from keras.models import Model
import tensorflow_datasets as tfds
from tqdm import tqdm

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
train_data, info = tfds.load('fashion_mnist', split='train', with_info=True)
test_data = tfds.load('fashion_mnist', split='test')

In [4]:
def format_image(data):
    image = data['image']
    image = tf.reshape(image, (-1, ))
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    return image, data['label']

Create a pipeline that formates images

In [5]:
train_data = train_data.map(format_image)
test_data = test_data.map(format_image)

In [6]:
batch_size = 64
train = train_data.shuffle(buffer_size=1024).batch(batch_size)
test = test_data.batch(batch_size)

## Define the model

In [7]:
def base_model():
    inputs = Input(shape=(784, ), name='input')
    x = Dense(units=64, activation='relu', name='dense1')(inputs)
    x = Dense(units=64, activation='relu', name='dense2')(x)
    x = Dense(units=10, activation='softmax', name='output')(x)
    return Model(inputs=inputs, outputs=x)

### Define loss and optimizer

In [8]:
loss_func = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam()

### Define Metrics

In [9]:
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
test_acc_metric = keras.metrics.SparseCategoricalAccuracy()

## Building training loop

In [10]:
@tf.function(reduce_retracing=False)
def apply_gradients(optimizer, model, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = loss_func(y, logits)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return logits, loss

In [11]:
def training_step():
    losses = []
    pbar = tqdm(total=len(list(enumerate(train))), position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')

    for step, (x_train_batch, y_train_batch) in enumerate(train):
        logits, loss = apply_gradients(optimizer, model, x_train_batch, y_train_batch)
        losses.append(loss)
        train_acc_metric(y_train_batch, logits)
        pbar.set_description(f'training loss for step {step}: {loss:0.4e}')
        pbar.update()
    return losses

### Also define validation loop

In [12]:
def validation_step():
    losses = []
    for x_test, y_test in test:
        logits = model(x_test)
        loss = loss_func(y_test, logits)
        losses.append(loss)
        test_acc_metric(y_test, logits)
    return losses

## Define training loop

In [13]:
model = base_model()

epochs = 10
epoch_val_losses, epoch_train_losses = [], []

for epoch in range(epochs):
    print(f'Start of the epoch {epoch:>3}')

    # Perform a training step
    losses = training_step()
    train_acc = train_acc_metric.result()

    # Perform validation step to get metrics
    val_losses = validation_step()
    val_acc = test_acc_metric.result()

    # Loss for each epoch should be mean over the steps
    train_loss_mean = np.mean(losses)
    val_loss_mean = np.mean(val_losses)

    # Store mean_loss
    epoch_train_losses.append(train_loss_mean)
    epoch_val_losses.append(val_loss_mean)

    print(f'''Epoch: {epoch + 1:>2} train_loss: {train_loss_mean:0.3e} train_acc: {train_acc_metric.result():0.2f} val_loss: {val_loss_mean:0.3e} val_acc:{test_acc_metric.result():0.2f}''')

    # reset metrics for the next epoch
    train_acc_metric.reset_states()
    test_acc_metric.reset_states()

Start of the epoch   0


training loss for step 937: 3.7706e-01: 100%|██████████████████████████| 938/938


Epoch:  1 train_loss: 5.448e-01 train_acc: 0.81 val_loss: 4.730e-01 val_acc:0.84
Start of the epoch   1


training loss for step 937: 4.0419e-01: 100%|██████████████████████████| 938/938


Epoch:  2 train_loss: 3.913e-01 train_acc: 0.86 val_loss: 4.227e-01 val_acc:0.85
Start of the epoch   2


...

Epoch:  8 train_loss: 2.791e-01 train_acc: 0.90 val_loss: 3.558e-01 val_acc:0.88
Start of the epoch   8


training loss for step 937: 2.7443e-01: 100%|██████████████████████████| 938/938


Epoch:  9 train_loss: 2.706e-01 train_acc: 0.90 val_loss: 3.512e-01 val_acc:0.88
Start of the epoch   9


training loss for step 937: 2.6668e-01: 100%|██████████████████████████| 938/938


Epoch: 10 train_loss: 2.603e-01 train_acc: 0.90 val_loss: 3.476e-01 val_acc:0.88
